In [1]:
import os
os.environ['AWS_PROFILE'] = 'admin'
os.environ['HAVEN_DATABASE'] = 'haven'

import pandas as pd 
import numpy as np 
import plotly.express as px 

from mirrorverse.utils import read_data_w_cache

In [ ]:
import numpy as np

matrix1 = np.array([[4, 4.001], [4.001, 4.002]])
inverted_matrix1 = np.linalg.inv(matrix1)
matrix2 = np.array([[4, 4.001], [4.001, 4.002001]])
inverted_matrix2 = np.linalg.inv(matrix2)

print(inverted_matrix1/inverted_matrix2)

a/b = m 

[[-2.99999924 -2.99999999]
 [-2.99999999 -2.99999999]]


In [9]:
A = np.array([[1, 2, 3], [3, 5, 3], [2, 8, 5]])
Ai = np.linalg.inv(A)
C = np.array([2, 2, 3])
np.matmul(Ai, C)

array([ 0.12, -0.08,  0.68])

In [11]:
np.matmul(A, np.matmul(Ai, C))

array([2., 2., 3.])

In [14]:
2 * 0.12 - 0.08 * 8 + 0.68 * 5

3.0000000000000004

In [16]:
(np.array([4, -4, 3, 2, -2]) * np.array([2, 3, 6, -5, 2])).sum()

0

In [2]:
model = '3_1_18'
run_id = 'dcdcf74abea8d96ff28901fcd9653fc783df9cdda20ed550adc8c2c38a11d896'
data = read_data_w_cache(
    f"select * from chinook_depth_inference_{model} where run_id = '{run_id}' and not _train"
)
print(data.shape)
data.head()

(1285984, 44)


,_individual,_decision,_choice,_selected,tag_key,h3_index,time,depth_bin,n_depth_bin,cos_moon,...,n_temperature,velocity_east,velocity_north,log_odds,odds,probability,experiment_name,run_id,_train,_partition
0,73,318541,2,False,159014b,840c539ffffffff,2017-05-10,50.0,0.2,-0.968880,...,0.328623,0.002695,0.016280,-1.173747,0.309206,0.145675,chinook-depth-3-1-18,dcdcf74abea8d96ff28901fcd9653fc783df9cdda20ed5...,False,1
1,73,318545,1,True,159014b,840c539ffffffff,2017-05-09,25.0,0.1,-0.894049,...,0.346807,-0.018762,0.016540,-0.871487,0.418329,0.717018,chinook-depth-3-1-18,dcdcf74abea8d96ff28901fcd9653fc783df9cdda20ed5...,False,1
2,73,318545,2,False,159014b,840c539ffffffff,2017-05-09,50.0,0.2,-0.894049,...,0.326194,0.000835,0.010227,-2.268380,0.103480,0.177365,chinook-depth-3-1-18,dcdcf74abea8d96ff28901fcd9653fc783df9cdda20ed5...,False,1
3,73,318545,3,False,159014b,840c539ffffffff,2017-05-09,75.0,0.3,-0.894049,...,0.327787,0.005103,0.010901,-2.786762,0.061620,0.105618,chinook-depth-3-1-18,dcdcf74abea8d96ff28901fcd9653fc783df9cdda20ed5...,False,1
4,73,318549,1,True,159014b,840c539ffffffff,2017-05-09,25.0,0.1,-0.894049,...,0.346807,-0.018762,0.016540,-0.146968,0.863321,0.776931,chinook-depth-3-1-18,dcdcf74abea8d96ff28901fcd9653fc783df9cdda20ed5...,False,1


In [3]:
data.groupby(['depth_bin'])[['_selected', 'probability']].mean()

,_selected,probability
depth_bin,,
25.0,0.464131,0.454763
50.0,0.166331,0.167608
75.0,0.134223,0.134586
100.0,0.106504,0.127291
150.0,0.129638,0.106225
200.0,0.048525,0.056552
250.0,0.012674,0.017843
300.0,0.006337,0.005192
400.0,0.002459,0.001708


In [5]:
data['log_likelihood'] = np.log(data['probability'])
data[data['_selected'] & data['_train']].groupby(['_individual'])['log_likelihood'].mean().reset_index()['log_likelihood'].mean()

-1.307528

In [ ]:
data['null_odds'] = 1.0
data['sum_null_odds'] = data.groupby(['_individual', '_decision'])['null_odds'].transform('sum')
data['null_probability'] = data['null_odds'] / data['sum_null_odds']
data.groupby(['depth_bin'])[['_selected', 'probability', 'null_probability']].mean()

,_selected,probability,null_probability
depth_bin,,,
25.0,0.464131,0.487248,0.172272
50.0,0.166331,0.161109,0.172272
75.0,0.134223,0.125124,0.162443
100.0,0.106504,0.117563,0.150641
150.0,0.129638,0.109963,0.134304
200.0,0.048525,0.047612,0.120577
250.0,0.012674,0.011076,0.106467
300.0,0.006337,0.002584,0.102181
400.0,0.002459,0.000603,0.100713


In [9]:
data['null_log_likelihood'] = np.log(data['null_probability'])
data[data['_selected'] & data['_train']].groupby(['_individual'])['null_log_likelihood'].mean().reset_index()['null_log_likelihood'].mean()

-1.8345387151294439

In [25]:
data[data['_selected'] & ~data['_train']][['probability', 'null_probability']].mean()

probability         0.302048
null_probability    0.179194
dtype: float64

In [26]:
color_discrete_map = {
    0.1: "#1b9e77",  # Green
    0.2: "#d95f02",  # Orange
    0.3: "#7570b3",  # Purple
    0.4: "#e7298a",  # Pink
    0.5: "#66a61e",  # Olive Green
    0.6: "#e6ab02",  # Yellow-Orange
    0.7: "#a6761d",  # Brown
    0.8: "#666666",  # Gray
    0.9: "#1f78b4",  # Blue
    1.0: "#a6cee3",  # Light Blue
}

data['orbit_radians'] = np.arctan2(data['sin_orbit'], data['cos_orbit'])
data['binned_orbit_radians'] = round(data['orbit_radians'] * 10) / (10)
df = data.groupby(['binned_orbit_radians', 'n_depth_bin'])['probability'].mean().reset_index()
px.line(
    df.sort_values('binned_orbit_radians'), x='binned_orbit_radians', y='probability', color='n_depth_bin', 
    color_discrete_map=color_discrete_map, title=f'model - {model}'

)

In [27]:
data['binned_n_mixed_layer_thickness'] = round(data['n_mixed_layer_thickness'] * 10) / (10)
df = data.groupby(['binned_n_mixed_layer_thickness', 'n_depth_bin'])['probability'].mean().reset_index()
px.line(
    df.sort_values('binned_n_mixed_layer_thickness'), x='binned_n_mixed_layer_thickness', y='probability', color='n_depth_bin', 
    color_discrete_map=color_discrete_map, title=f'model - {model}'
)

In [28]:
data['binned_n_salinity'] = round(data['n_salinity'] * 10) / (10)
df = data.groupby(['binned_n_salinity', 'n_depth_bin'])['probability'].mean().reset_index()
px.line(
    df.sort_values('binned_n_salinity'), x='binned_n_salinity', y='probability', color='n_depth_bin', 
    color_discrete_map=color_discrete_map, title=f'model - {model}'
)

In [29]:
data['month'] = data['time'].apply(lambda x: x.month)
data['sun_radians'] = np.arctan2(data['sin_sun'], data['cos_sun'])
data['binned_sun_radians'] = round(data['sun_radians'] * 10) / (10)
df = data.groupby(['month', 'binned_sun_radians', 'n_depth_bin'])['probability'].mean().reset_index()
px.line(
    df.sort_values('binned_sun_radians'), x='binned_sun_radians', y='probability', color='n_depth_bin', 
    color_discrete_map=color_discrete_map, title=f'model - {model}', facet_row='month', height=1600
)